In [1]:
import json
!pip install datasets
from datasets import load_dataset

In [2]:
dataset_name = 'imdb'
raw_datasets = load_dataset(dataset_name)

/projects/metis2/atharvak/miniconda3/envs/11711_env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Found cached dataset imdb (/home/atharvak/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
import pandas as pd
import os

In [5]:
# remove samples with label -1
raw_datasets['train'] = raw_datasets['train'].filter(lambda x:x['label']!=-1)
raw_datasets['test'] = raw_datasets['test'].filter(lambda x:x['label']!=-1)

df_train = pd.DataFrame()
if dataset_name!= "sst2":
    ids = list(range(len(raw_datasets['train'])))
    raw_datasets['train'] = raw_datasets['train'].add_column('idx',ids)

    df_train["idx"] = raw_datasets["train"]["idx"]
    df_train["text"] = raw_datasets["train"]["text"]
    df_train["label"] = raw_datasets["train"]["label"]
    
    df_test = pd.DataFrame()
    df_test["text"] = raw_datasets["test"]["text"]
    df_test["label"] = raw_datasets["test"]["label"]

#     print(df_train.head())
#     print(os.getcwd())
    df_train.to_csv(f"/projects/metis2/atharvak/Data_Cartography/datasets/{dataset_name}_train.csv", header = True, index = False)
    df_test.to_csv(f"/projects/metis2/atharvak/Data_Cartography/datasets/{dataset_name}_test.csv", header = True, index = False)

else:
    df_train = pd.DataFrame()
    df_train["idx"] = raw_datasets["train"]["idx"]
    df_train["text"] = raw_datasets["train"]["sentence"]
    df_train["label"] = raw_datasets["train"]["label"]
    
    df_test = pd.DataFrame()
    df_test["text"] = raw_datasets["validation"]["sentence"]
    df_test["label"] = raw_datasets["validation"]["label"]
#     raw_datasets.save_to_disk(f"datasets/{dataset_name}/with_idx")

#     df_train.to_csv(os.getcwd()+f"{dataset_name}_train.csv", header = True, index = False)
#     df_test.to_csv(os.getcwd()+f"/datasets/{dataset_name}/test.csv", header = True, index = False)

Loading cached processed dataset at /home/atharvak/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-9a103be8427e9b02.arrow
Loading cached processed dataset at /home/atharvak/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-38445888b5967e2f.arrow
Loading cached processed dataset at /home/atharvak/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-d2adb714cc80a889.arrow


In [6]:
import json
f = open(f'/projects/metis2/atharvak/Data_Cartography/dy_log/{dataset_name}/roberta-base/three_regions_data_indices.json')
data = json.load(f)
new_dict = {}
category = []
for line in data['hard']:
    new_dict[line] = 'hard'
for line in data['easy']:
    new_dict[line] = 'easy'
for line in data['ambiguous']:
    new_dict[line] = 'ambiguous'

for line in df_train['idx']:
    if line not in new_dict.keys():
        category.append('none')
    else:
        category.append(new_dict[line])

print (category[:50])
df_train['category'] = category

df_train.to_csv(f"/projects/metis2/atharvak/Data_Cartography/datasets/{dataset_name}/{dataset_name}_categorized.csv", header = True, index = False)

['ambiguous', 'easy', 'easy', 'easy', 'none', 'easy', 'none', 'none', 'easy', 'none', 'none', 'easy', 'none', 'ambiguous', 'ambiguous', 'easy', 'easy', 'easy', 'easy', 'easy', 'ambiguous', 'none', 'easy', 'ambiguous', 'easy', 'none', 'easy', 'none', 'easy', 'easy', 'easy', 'none', 'ambiguous', 'easy', 'easy', 'none', 'easy', 'easy', 'easy', 'ambiguous', 'easy', 'none', 'none', 'none', 'none', 'easy', 'easy', 'easy', 'ambiguous', 'none']


In [7]:
df = pd.read_csv('/projects/metis2/atharvak/Data_Cartography/datasets/imdb/imdb_categorized.csv')
df

,idx,text,label,category
0,0,I rented I AM CURIOUS-YELLOW from my video sto...,0,ambiguous
1,1,"""I Am Curious: Yellow"" is a risible and preten...",0,easy
2,2,If only to avoid making this type of film in t...,0,easy
3,3,This film was probably inspired by Godard's Ma...,0,easy
4,4,"Oh, brother...after hearing about this ridicul...",0,none
...,...,...,...,...
24995,24995,A hit at the time but now better categorised a...,1,ambiguous
24996,24996,I love this movie like no other. Another time ...,1,none
24997,24997,This film and it's sequel Barry Mckenzie holds...,1,none
24998,24998,'The Adventures Of Barry McKenzie' started lif...,1,ambiguous


In [8]:
df['category'].value_counts()

ambiguous    8250
easy         8250
none         8042
hard          458
Name: category, dtype: int64

In [73]:
df_e = df_train.loc[df_train['category'] == "easy"]
df_easy = pd.DataFrame()
df_easy['text'] = df_e['text']
df_easy['label'] = df_e['label']
# df_easy.to_csv(os.getcwd()+f"/datasets/{dataset_name}/{dataset_name}_easy.csv", header = True, index = False)

In [74]:
df_h = df_train.loc[df_train['category'] == "hard"]
df_hard = pd.DataFrame()
df_hard['text'] = df_h['text']
df_hard['label'] = df_h['label']
# df_hard.to_csv(os.getcwd()+f"/datasets/{dataset_name}/{dataset_name}_hard.csv", header = True, index = False)

In [75]:
df_a = df_train.loc[df_train['category'] == "ambiguous"]
df_ambi = pd.DataFrame()
df_ambi['text'] = df_a['text']
df_ambi['label'] = df_a['label']
# df_ambi.to_csv(os.getcwd()+f"/datasets/{dataset_name}/{dataset_name}_ambi.csv", header = True, index = False)

In [76]:
df_t1= df_train.loc[df_train['category'] == "easy"]
df_t2 = df_train.loc[df_train['category'] == "ambiguous"]

df_easy= pd.DataFrame()
df_easy['text'] = df_t1['text']
df_easy['label'] = df_t1['label']
df_hard = pd.DataFrame()
df_hard['text'] = df_t2['text']
df_hard['label'] = df_t2['label']

df_easy_hard = pd.DataFrame()
df_easy_hard = pd.concat([df_easy, df_hard])

df_easy_hard.sample(frac=1)
df_easy_hard.to_csv(os.getcwd()+f"/datasets/{dataset_name}/{dataset_name}_easy_ambi.csv", header = True, index = False)

In [64]:
df_easy_hard

,text,label
2,that loves its characters and communicates som...,1
15,the greatest musicians,1
17,with his usual intelligence and subtlety,1
19,swimming is above all about a young woman 's f...,1
22,a smile on your face,1
...,...,...
67343,showing off his doctorate,1
67345,"anguish , anger and frustration",0
67346,"at achieving the modest , crowd-pleasing goals...",1
67347,a patient viewer,1


In [65]:
df_train["category"].value_counts()

ambiguous    22225
easy         22225
none         21091
hard          1808
Name: category, dtype: int64

In [22]:
df_t1

,idx,text,label,category
2,2,that loves its characters and communicates som...,1,easy
15,15,the greatest musicians,1,easy
17,17,with his usual intelligence and subtlety,1,easy
19,19,swimming is above all about a young woman 's f...,1,easy
22,22,a smile on your face,1,easy
...,...,...,...,...
67328,67328,is nothing short of a great one .,1,easy
67333,67333,comedy ensemble,1,easy
67335,67335,brilliant movie,1,easy
67338,67338,"polished ,",1,easy


In [69]:
pd.read_csv(os.getcwd()+f"/datasets/{dataset_name}/{dataset_name}_easy_hard.csv")

,text,label
0,I'm writing this review to give you a heads up...,0
1,Wing sauce is like water. Pretty much a lot of...,0
2,I drove by yesterday to get a sneak peak. It ...,1
3,After waiting for almost 30 minutes to trade i...,0
4,I got 'new' tires from them and within two wee...,0
...,...,...
184795,When Corey sent the email about tickets being ...,1
184796,"Wow, really? Bought one item from them, a ham...",0
184797,"Long line, inefficient staff. Maybe my expecta...",0
184798,Professional \nFriendly\nOn time AND affordabl...,1
